In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import scipy.optimize as sop
from matplotlib import pyplot as plt
from fenix_alpha_vantage.alpha_stock_timeseries import AlphaStockTimeSeries

In [4]:
class StockForecastType:
    pass

class PastPercentualReturnForecastType(StockForecastType):
    def __init__(self, look_back_days, percentual_returns_window_days, average_trade_return_window_days):
        self.look_back_days = look_back_days # Number of days to consider as a data input
        self.percentual_returns_window_days = percentual_returns_window_days
        self.average_trade_return_window_days = average_trade_return_window_days

# Heuristics Definition

In [5]:
reference_date = '2021-01-01'

to_consider = ['ABEV3.SAO', 
               'LREN3.SAO',
               'AZUL4.SAO', 
               'LIGT3.SAO',
               'SHOW3.SAO',
               'PETR4.SAO',
               'JBSS3.SAO',
               'OIBR3.SAO',
               'BBDC4.SAO',
               'NTCO3.SAO']
# to_consider = ['BBAS3.SAO',
#                'BBDC4.SAO',
#                'BPAC11.SAO',
#                'ITUB4.SAO',
#                'SHOW3.SAO']

forecast_type = PastPercentualReturnForecastType(look_back_days=150,
                                                 percentual_returns_window_days=1,
                                                 average_trade_return_window_days=30)


# Calendar

In [6]:
reference_date = pd.to_datetime(reference_date) - pd.Timedelta(days=1)
df_calendar = pd.DataFrame(
    {"date": pd.date_range(
        reference_date - pd.Timedelta(days=forecast_type.look_back_days),
        reference_date
    )})

#-- dia anterior menos a janela
df_calendar["day_of_week"] = df_calendar['date'].dt.isocalendar().day
df_calendar["week_of_year"] = df_calendar['date'].dt.isocalendar().week
df_calendar["year"] = df_calendar['date'].dt.isocalendar().year

# Data ETL

### Getting data from Alpha Vantage

In [7]:
timeseries = AlphaStockTimeSeries()

In [10]:
daily_api_dfs = {}
for ticker in to_consider:
    daily_api_dfs[ticker] = timeseries.df_time_series_daily(ticker=ticker, use_vpn=True, adjusted=True)

### Aggregating data to one table only

In [11]:
# Backups
daily_dfs = daily_api_dfs.copy()

In [13]:
ticker_close_column_names = []
close_column_name = 'close'
df_closings = df_calendar.copy()

# One table for all stocks
for stock in daily_dfs:
    # Getting Stock name
    prefix = stock.lower()[0:5]+'_'
    #Selecting only the columns of interest and renaming
    daily_dfs[stock] = daily_dfs[stock][['date', 'adjusted_close']]
    daily_dfs[stock] = daily_dfs[stock].rename({'adjusted_close':close_column_name}, axis='columns')
    # Adding Prefix to all columns
    daily_dfs[stock] = daily_dfs[stock].add_prefix(prefix)
    ticker_close_column_names.append(prefix+close_column_name)
    # Joining with calendar
    df_closings = df_closings.merge(daily_dfs[stock], how='inner', left_on='date', right_on=prefix+'date')
    del df_closings[prefix+'date']
df_closings = df_closings.set_index('date')

### Removing date details

In [14]:
df_closings = df_closings[ticker_close_column_names]

# Dataframe being used

In [15]:
df_closings

,abev3_close,lren3_close,azul4_close,ligt3_close,show3_close,petr4_close,jbss3_close,oibr3_close,bbdc4_close,ntco3_close
date,,,,,,,,,,
2020-08-03,13.2935,40.5587,20.1,18.0479,2.65,21.0902,21.7766,1.62,20.0564,46.61
2020-08-04,12.9929,39.2273,19.82,17.1211,2.66,21.0708,21.2539,1.55,19.637,46.74
2020-08-05,12.7796,40.4792,20.16,17.0724,2.63,22.4253,21.1087,1.62,19.7274,47.81
2020-08-06,12.8572,42.5757,21.45,17.1114,2.69,22.464,21.341,1.56,19.8631,48.03
2020-08-07,12.9056,42.9434,21.01,16.7894,2.8,22.048,20.9732,1.56,19.7455,46.79
...,...,...,...,...,...,...,...,...,...,...
2020-12-22,15.4364,43.0417,35.27,22.3599,3.81,26.3918,23.2864,2.11,24.5745,50.05
2020-12-23,15.4265,43.6002,37.7,23.5306,3.84,27.04,23.0154,2.16,25.1368,51.69
2020-12-28,15.6952,43.8495,37.77,23.8135,3.89,27.2625,23.0831,2.22,25.3816,52.0


# Calculations

In [16]:
print("(stock price / previous stock price) - 1")
returns_window = 1
df_percentual_returns = df_closings.pct_change(periods=forecast_type.percentual_returns_window_days)

(stock price / previous stock price) - 1


## Standardizing Return Values

In [17]:
# Average return window -> Trade Window
df_average_return = df_percentual_returns.rolling(forecast_type.average_trade_return_window_days).mean()

## Returns Covariance

In [18]:
df_returns_covariance_matrix = df_percentual_returns.astype(float).cov()

# Portfolio Optimization

In [19]:
# Ponto de atencao
def check_sum_to_one(weights):
    return np.sum(weights)-1

def check_inequality(weights):
    return weights

In [20]:
# Starting values -> Fazer mais dinamico
weights_start = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

# Can be anywhere from 0 to 1 -> We can set our hardcoded % of a stock here
bounds = ((0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1))

In [21]:
# Constraints from our minimization
constraints = ([
    {'type':'eq', 'fun':check_sum_to_one},
    {'type':'ineq', 'fun':check_inequality}
])

In [22]:
def negative_sr(weights):
    weights = np.array(weights)
    
    ret = np.sum(df_average_return[-1:].mean()*weights)
    volatility = np.dot(weights.T, np.dot(df_returns_covariance_matrix, weights))
    
    sr = volatility/ret
    return sr

In [23]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
optimal_weights = sop.minimize(fun=negative_sr,
                               x0=weights_start, 
                               method='SLSQP',
                               bounds=bounds,
                               constraints=constraints
                              )

In [24]:
optimal_weights

     fun: 0.04463152454702949
     jac: array([0.04948486, 0.10289718, 0.05727246, 0.04454546, 0.0444776 ,
       0.04426819, 0.04502336, 0.04429378, 0.04470138, 0.04448176])
 message: 'Optimization terminated successfully'
    nfev: 154
     nit: 14
    njev: 14
  status: 0
 success: True
       x: array([0.00000000e+00, 1.34476165e-17, 2.40376394e-18, 4.85411369e-02,
       1.54850022e-01, 2.20091430e-01, 3.62533246e-01, 4.80971097e-02,
       3.19590560e-02, 1.33928000e-01])

In [25]:
if optimal_weights.success:
    print("Optimal Allocation")
    print("")
    for ticker, value in zip(ticker_close_column_names, list(optimal_weights.x.astype(float)*100)):
        print(f'{ticker[:5]} - {round(value,3)}%')
else:
    print('Something Went wrong...')

Optimal Allocation

abev3 - 0.0%
lren3 - 0.0%
azul4 - 0.0%
ligt3 - 4.854%
show3 - 15.485%
petr4 - 22.009%
jbss3 - 36.253%
oibr3 - 4.81%
bbdc4 - 3.196%
ntco3 - 13.393%


---

# Correlation

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
correlation_matrix = average_return_df.corr()

In [ ]:
plt.rcParams['figure.figsize'] = (10,10)
sns.set(font_scale=1.4)
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm',  fmt='.1g', vmin = -1, vmax = 1)

# Decomposing

In [ ]:
same_period.tail()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
multiplicable_result=seasonal_decompose(same_period['abev3_adjusted close'], model='multiplicable', period=12)
additive_result=seasonal_decompose(same_period['abev3_adjusted close'], model='additive', period=12)

In [ ]:
additive_result.trend.plot()

In [ ]:
additive_result.plot()

# Random Portfolio Allocations

In [ ]:
number_of_random_portfolios = 5
random_portfolio_weights = list()

portfolio_return = []
portfoli_risk = []
sharp_ratio = []

for ticker in range(5):
    
    # Creating rando portfolio distributions
    random_weights = np.random.rand(len(to_consider))   
    random_portfolio_weights.append(np.round(random_weights/np.sum(random_weights),3))
    
    

In [ ]:
df.pct_change()